In [ ]:
!pip install --no-warn-conflicts -q --upgrade xgboost

This is a fun competition for testing ideas and probably **Overfiting**! Using baseline predictions for Catboost and XGboost might improve their performance. I'm testing this idea in this notebook. XGboost parameters are from [https://www.kaggle.com/tunguz/tps-02-21-feature-importance-with-xgboost-and-shap](https://www.kaggle.com/tunguz/tps-02-21-feature-importance-with-xgboost-and-shap) and the LightGBM model is from [https://www.kaggle.com/awwalmalhi/extreme-fine-tuning-lgbm-using-7-step-training](https://www.kaggle.com/awwalmalhi/extreme-fine-tuning-lgbm-using-7-step-training)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
from functools import partial
from xgboost import DMatrix
import lightgbm as lgbm
import xgboost as xgb 
import seaborn as sns
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')

In [ ]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train.target
X_test = test.drop(['id'], axis=1)

In [ ]:
cat_cols = [feature for feature in train.columns if 'cat' in feature]
cont_cols = [feature for feature in train.columns if 'con' in feature]

for feature in cat_cols:
    le = LabelEncoder()
    le.fit(train[feature])
    X_train[feature] = le.transform(X_train[feature])
    X_test[feature] = le.transform(X_test[feature])


In [ ]:
seed = 0
n_splits = 6
shuffle=True
iterations = 50000
early_stopping_rounds = 400
verbose_eval = 0
baseline_rounds = 1
cb_learning_rate = 0.006
xgb_learning_rate = 0.01

In [ ]:
split = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

In [ ]:
cb_params = {'iterations':iterations,
             'learning_rate':cb_learning_rate,
             'depth':7,
             'bootstrap_type':'Bernoulli',
             'random_strength':1,
             'min_data_in_leaf':10,
             'l2_leaf_reg':3,
             'loss_function':'RMSE', 
             'eval_metric':'RMSE',
             'random_seed':seed,
             'grow_policy':'Depthwise',
             'max_bin':1024, 
             'model_size_reg': 0,
             'task_type': 'GPU',
             'od_type':'IncToDec',
             'od_wait':100,
             'metric_period':500,
             'verbose':verbose_eval,
             'subsample':0.8,
             'od_pval':1e-10,
             'max_ctr_complexity': 8,
             'has_time': False,
             'simple_ctr' : 'FeatureFreq',
             'combinations_ctr': 'FeatureFreq'
            }

xgb_params= {'objective': 'reg:squarederror',
             'max_depth': 6,
             'eta': xgb_learning_rate,
             'colsample_bytree': 0.4,
             'subsample': 0.6,
             'reg_alpha' : 6,
             'min_child_weight': 100,
             'n_jobs': 2,
             'seed': 2001,
             'tree_method': 'gpu_hist',
             'gpu_id': 0,
             'predictor': 'gpu_predictor',
            }

lgbm_params = {'max_depth': 16,
               'subsample': 0.8032697250789377, 
               'colsample_bytree': 0.21067140508531404,
               'learning_rate': 0.009867383057779643,
               'reg_lambda': 10.987474846877767,
               'reg_alpha': 17.335285595031994,
               'min_child_samples': 31, 
               'num_leaves': 66,
               'max_bin': 522,
               'cat_smooth': 81,
               'cat_l2': 0.029690334194270022,
               'metric': 'rmse',
               'n_jobs': -1, 
               'verbose':-1,
               'n_estimators': iterations
              }


In [ ]:
preds_list = []
oof_cb = np.zeros((len(train)))
oof_xgb = np.zeros((len(train)))
oof_cbx = np.zeros((len(train)))
oof_xgbx = np.zeros((len(train)))
oof_lgb = np.zeros((len(train)))
oof_lgb_incremental = np.zeros((len(train)))
stack_oof = np.zeros((len(train)))
stack_preds = np.zeros((len(test)))

for fold, (train_idx, val_idx) in enumerate(split.split(X_train)):
    print(f'Fold {fold+1}')
    X_tr = X_train.iloc[train_idx]
    X_val = X_train.iloc[val_idx]
    y_tr = y_train.iloc[train_idx]
    y_val = y_train.iloc[val_idx]
    fold_stack_oof = np.zeros((len(X_val), 6))
    fold_stack_preds = np.zeros((len(test), 6))
    ptrain = Pool(data=X_tr, label=y_tr, cat_features=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    pvalid = Pool(data=X_val, label=y_val, cat_features=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    ptest = Pool(data=X_test, cat_features=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    CModel = CatBoostRegressor(**cb_params)
    CModel.fit(ptrain,
               eval_set=pvalid,
               use_best_model=True,
               early_stopping_rounds=early_stopping_rounds)
    temp_fold_preds = CModel.predict(pvalid)
    oof_cb[val_idx] = temp_fold_preds
    first_cb_rmse = mean_squared_error(y_val, temp_fold_preds, squared=False)
    print(f'RMSE of CB model is {first_cb_rmse}')
    baseline_preds_tr_cb = CModel.predict(ptrain)
    baseline_preds_vl_cb = temp_fold_preds
    test_preds_cb = CModel.predict(ptest)   
    fold_stack_oof[:,0] = temp_fold_preds
    fold_stack_preds[:,0] = test_preds_cb
    xtrain = DMatrix(data=X_tr, label=y_tr, nthread=2)
    xvalid = DMatrix(data=X_val, label=y_val, nthread=2)
    xtest = DMatrix(data=X_test, nthread=2)
    XModel = xgb.train(xgb_params, xtrain,
                       evals=[(xvalid,'validation')],
                       verbose_eval=verbose_eval,
                       early_stopping_rounds=early_stopping_rounds,
                       xgb_model=None,
                       num_boost_round=iterations)
    temp_fold_preds = XModel.predict(xvalid)
    oof_xgb[val_idx] = temp_fold_preds
    first_xgb_rmse = mean_squared_error(y_val, temp_fold_preds, squared=False)
    print(f'RMSE of XGB model is {first_xgb_rmse}')
    baseline_preds_tr_xgb = XModel.predict(xtrain)
    baseline_preds_vl_xgb = temp_fold_preds
    test_preds_xgb = XModel.predict(xtest)
    fold_stack_oof[:,1] = temp_fold_preds
    fold_stack_preds[:,1] = test_preds_xgb
    ltrain = lgbm.Dataset(X_tr, label=y_tr, init_score=None, categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], free_raw_data=False)
    lvalid = lgbm.Dataset(X_val, label=y_val, init_score=None, categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], free_raw_data=False)
    ltest =  lgbm.Dataset(X_test, label=y_val, init_score=None, categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], free_raw_data=False)
    LModel = lgbm.train(lgbm_params,
                        train_set=ltrain,
                        num_boost_round=iterations,
                        valid_sets=lvalid, 
                        init_model=None,
                        early_stopping_rounds=early_stopping_rounds,
                        verbose_eval=verbose_eval)           
    temp_fold_preds = LModel.predict(X_val)
    oof_lgb[val_idx] = temp_fold_preds
    fold_stack_oof[:,2] = temp_fold_preds
    fold_stack_preds[:,2] = LModel.predict(X_test)
    first_lgb_rmse = mean_squared_error(y_val, temp_fold_preds, squared=False)
    print(f'RMSE of LGBM model is {first_lgb_rmse}')
    params = lgbm_params.copy()     
    params.update({'learning_rate': 0.003})
    for i in range(1, 9):
        if i > 2:                      
            params['reg_lambda'] *= 0.9
            params['reg_alpha']  *= 0.9
            params['num_leaves'] += 40                   
        
        LModel = lgbm.train(lgbm_params,
                            train_set=ltrain,
                            num_boost_round=iterations,
                            valid_sets=lvalid, 
                            init_model=LModel,
                            early_stopping_rounds=early_stopping_rounds,
                            verbose_eval=verbose_eval)           
    temp_fold_preds = LModel.predict(X_val)
    oof_lgb_incremental[val_idx] = temp_fold_preds
    second_lgb_rmse = mean_squared_error(y_val, temp_fold_preds, squared=False)
    print(f'RMSE of LGBM model is {second_lgb_rmse}')
    print(f'LGBM improvement using Incremental Improvements {first_lgb_rmse - second_lgb_rmse}')
    baseline_preds_tr_lgb = LModel.predict(X_tr)
    baseline_preds_vl_lgb = temp_fold_preds
    test_preds_lgb = LModel.predict(X_test)
    fold_stack_oof[:,3] = temp_fold_preds
    fold_stack_preds[:,3] = test_preds_lgb
    
    baseline_train = (baseline_preds_tr_xgb+baseline_preds_tr_lgb+baseline_preds_tr_cb)/3
    baseline_valid = (baseline_preds_vl_xgb+baseline_preds_vl_lgb+baseline_preds_vl_cb)/3
    baseline_test = (test_preds_xgb+test_preds_lgb+test_preds_cb)/3
    
    for baseline in range(baseline_rounds):
        ptrain = Pool(data=X_tr, label=y_tr, cat_features=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], baseline=baseline_train)
        pvalid = Pool(data=X_val, label=y_val, cat_features=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], baseline=baseline_valid)
        ptest = Pool(data=X_test, cat_features=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], baseline=baseline_test)
        cb_params_ = cb_params.copy()
        cb_params_.update({'learning_rate': cb_learning_rate*(1/(baseline+1))})
        CModel = CatBoostRegressor(**cb_params_)
        CModel.fit(ptrain, 
                   eval_set=pvalid,
                   use_best_model=True,
                   early_stopping_rounds=early_stopping_rounds)
        temp_fold_preds = CModel.predict(pvalid)
        oof_cbx[val_idx] = temp_fold_preds
        second_cb_rmse = mean_squared_error(y_val, temp_fold_preds, squared=False)
        print(f'RMSE of CB model with baseline round {baseline+1} {second_cb_rmse}')   
        baseline_train = CModel.predict(ptrain)
        baseline_valid = CModel.predict(pvalid)
        baseline_test = CModel.predict(ptest)
        if baseline == baseline_rounds - 1:
            fold_stack_oof[:,4] = temp_fold_preds
            fold_stack_preds[:,4] = baseline_test
        
        xtrain = DMatrix(data=X_tr, label=y_tr, base_margin=baseline_train)
        xvalid = DMatrix(data=X_val, label=y_val, base_margin=baseline_valid)
        xtest =  DMatrix(data=X_test, base_margin=baseline_test)
        xgb_params_ = xgb_params.copy()
        xgb_params_.update({'learning_rate': xgb_learning_rate*(1/(baseline+1))})
        XModel = xgb.train(xgb_params_, xtrain,
                           evals=[(xvalid,'validation')],
                           verbose_eval=verbose_eval,
                           early_stopping_rounds=early_stopping_rounds,
                           xgb_model=None,
                           num_boost_round=iterations)
        temp_fold_preds = XModel.predict(xvalid)
        oof_xgbx[val_idx] = temp_fold_preds
        baseline_train = XModel.predict(xtrain)
        baseline_valid = temp_fold_preds
        baseline_test = XModel.predict(xtest)
        if baseline == baseline_rounds - 1:
            fold_stack_oof[:,5] = temp_fold_preds
            fold_stack_preds[:,5] = baseline_test
        second_xgb_rmse = mean_squared_error(y_val, temp_fold_preds, squared=False)
        print(f'RMSE of XGB model with baseline round {baseline+1} {second_xgb_rmse}')
        print(f'CB Improvement  using Baseline round {baseline+1}: {first_cb_rmse - second_cb_rmse}')
        print(f'XGB Improvement using Baseline round {baseline+1}: {first_xgb_rmse - second_xgb_rmse}')
        first_cb_rmse = second_cb_rmse
        first_xgb_rmse = second_xgb_rmse
    
    stacker = RidgeCV().fit(fold_stack_oof, y_val)
    temp_stack_fold_pred = stacker.predict(fold_stack_oof)
    stack_oof[val_idx] = temp_stack_fold_pred
    stack_rmse = mean_squared_error(y_val, temp_stack_fold_pred, squared=False)
    print(f'RMSE of stack model  {stack_rmse}')
    stack_preds += stacker.predict(fold_stack_preds)/n_splits
    print('-' * 100)
    print('',end='\n')
    

Lightgbm also has an **init_score** in it's **Dataset** but it's not working the same way as Catboost and XGBoost and providing high score predictions would lead to interference with model's learning process(It's based on my tests and maybe somebody could make it work!)

In [ ]:
first_cb_rmse = mean_squared_error(y_train, oof_cb, squared=False)
first_xgb_rmse = mean_squared_error(y_train, oof_xgb, squared=False)
first_lgb_rmse = mean_squared_error(y_train, oof_lgb, squared=False)
second_cb_rmse = mean_squared_error(y_train, oof_cbx, squared=False)
second_xgb_rmse = mean_squared_error(y_train, oof_xgbx, squared=False)
second_lgb_rmse = mean_squared_error(y_train, oof_lgb_incremental, squared=False)
print(f'RMSE for CB model is {first_cb_rmse}')
print(f'RMSE for XGB model is {first_xgb_rmse}')
print(f'RMSE for LGBM model is {first_lgb_rmse}')
print(f'RMSE for CB model with XGB baseline is {second_cb_rmse}')
print(f'RMSE for XGB model with CB baseline is {second_xgb_rmse}')
print(f'RMSE for LGBM model with Incremental Improvement is {second_lgb_rmse}')
print(f'RMSE for CB and XGB blend is {mean_squared_error(y_train, (oof_cbx+oof_xgbx)/2, squared=False)}')
print(f'RMSE for CB, XGB and LGBM blend is {mean_squared_error(y_train, (oof_cbx+oof_xgbx+oof_lgb_incremental)/3, squared=False)}')
print(f'RMSE for CB, XGB and LGBM stack is {mean_squared_error(y_train, stack_oof, squared=False)}')

Final predictions are a simple **stack** of **XGBoost**, **LightGBM** and **Catboost** models.

In [ ]:
preds = np.mean(preds_list, axis=0)
submission_mean = pd.DataFrame({'id':test.id,'target':stack_preds})
submission_mean.to_csv('submission_mean.csv', index=False)
submission_mean.head()

Another nice idea for more **overfit**. Blending with [**top public blend**](https://www.kaggle.com/somayyehgholami/comparative-method-tabular-feb-301)!

In [ ]:
sub_best = pd.read_csv('../input/comparative-method-tabular-feb-301/submission.csv')
sub_final = submission_mean.copy()
sub_final['target'] = sub_best.target * 0.9 + submission_mean.target * 0.1
sub_final.to_csv('blend_mean.csv', index=False)
sub_final.head()